In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
#连接谷歌云盘
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


wandb    : key:  dd794edc07382b51015cedce80ab05663924693b

In [ ]:
#数据可视化库
!pip install wandb
!wandb login

     |████████████████████████████████| 1.8 MB 4.3 MB/s 
     |████████████████████████████████| 181 kB 80.9 MB/s 
     |████████████████████████████████| 144 kB 88.0 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=94bcb10de461cef0d807747df2a85a89d53b4c8ed67a6b53d550f97f7cb4197c
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
!pip install transformers #huggingface库

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
#import wandb
from transformers import TrainingArguments, Trainer
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.model_selection import train_test_split
#模型列表以及可调参数、模型选择、最大长度
model_list = ['hfl/chinese-bert-wwm-ext',
'hfl/chinese-roberta-wwm-ext',
'hfl/chinese-bert-wwm',
'bert-base-chinese',
'hfl/rbt3',
]
model_name = model_list[4]
max_length = 100
dataset_name = '/content/drive/MyDrive/nlp_dataset/NLP_cleaned.csv'
wandb.init(project="bert-hit", entity="zib",name = model_name)

NameError: ignored

In [ ]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).
 
    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf
 
        tf.random.set_seed(seed)
 
set_seed(1)

In [ ]:
#数据集overview 总览大致长度分布
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
dataset = pd.read_csv(dataset_name)
dataset = dataset.dropna()
print(dataset['simplified_review'].str.len().describe())


count    13230.000000
mean        64.538776
std         48.189132
min          1.000000
25%         24.000000
50%         52.000000
75%         96.000000
max        365.000000
Name: simplified_review, dtype: float64


In [ ]:
#数据集划分
def read_comment(test_size=0.2):
  # download & load 20newsgroups dataset from sklearn's repos
  dataset = pd.read_csv(dataset_name)
  dataset = dataset.dropna()
  # split into training & testing a return data as well as label names
  return train_test_split(dataset['simplified_review'].values.tolist(),dataset['label'].values.tolist(), random_state=2018, test_size=test_size), dataset['label'].unique()
  
# call the function
(train_texts, valid_texts, train_labels, valid_labels), target_names = read_comment()

NameError: ignored

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(model_name)

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/828 [00:00<?, ?B/s]

In [ ]:
sentences = dataset.simplified_review.values
a = [ ]
for i in sentences:
  if len(i)<=10:
    a.append(i)

In [ ]:
tokenizer(a[0],return_tensors = 'pt')

{'input_ids': tensor([[ 101, 4867, 2207, 1987, 1987, 4495, 3189, 2571,  727, 1506, 1506,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
# tokenize the dataset, truncate when passed `max_length`, 
# and pad with 0's when less than `max_length`

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)


In [ ]:
class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = NewsGroupsDataset(train_encodings, train_labels)
valid_dataset = NewsGroupsDataset(valid_encodings, valid_labels)

In [ ]:
# load the model and pass to CUDA
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names)).to("cuda")

loading configuration file https://huggingface.co/hfl/rbt3/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/cbdc33938a9de5780d1f5c2b51aa1b0e8950089af09bdcc2adb2e502bef037aa.ed07afe1a0de762dea37c369ce59a78dba41a2845cf524299d856b92017e1203
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_head

Downloading:   0%|          | 0.00/149M [00:00<?, ?B/s]

storing https://huggingface.co/hfl/rbt3/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/51b739170f301b83ef254708db50e9f6885a0382f53f77410b56855e125e4801.79022cd0a80ef847554ccf94e41b2238a87ea5302fc4fbadbf9cdc521c781d20
creating metadata file for /root/.cache/huggingface/transformers/51b739170f301b83ef254708db50e9f6885a0382f53f77410b56855e125e4801.79022cd0a80ef847554ccf94e41b2238a87ea5302fc4fbadbf9cdc521c781d20
loading weights file https://huggingface.co/hfl/rbt3/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/51b739170f301b83ef254708db50e9f6885a0382f53f77410b56855e125e4801.79022cd0a80ef847554ccf94e41b2238a87ea5302fc4fbadbf9cdc521c781d20
Some weights of the model checkpoint at hfl/rbt3 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.pred

In [ ]:
from sklearn.metrics import accuracy_score ,precision_score ,recall_score ,f1_score
#定义衡量指标参数
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  pre = precision_score(labels, preds,average='macro')
  recall = recall_score(labels, preds,average='macro')
  f1 = f1_score(labels, preds,average='macro')
  return {
      'accuracy': acc,
      'precision': pre,
      'recall':recall,
      'F1':f1,
  }

In [ ]:
#训练所需要的参数
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    learning_rate=2e-5,             #learning rate
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    metric_for_best_model='accuracy',
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=200,               # log & save weights each logging_steps
    save_steps=200,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
    report_to="wandb",          #parse the result to the wandb to visuliaze
)

using `logging_steps` to initialize `eval_steps` to 200
PyTorch: setting up devices


In [ ]:
#构建训练任务
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [ ]:
# 训练模型
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10584
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3310
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
200,1.216900,0.803980,0.651550,0.610892,0.603387,0.589688
400,0.653900,0.527003,0.776644,0.750146,0.742063,0.740135
600,0.511400,0.475735,0.803099,0.786726,0.776347,0.773460
800,0.418700,0.461198,0.828798,0.810735,0.803290,0.805522
1000,0.396500,0.421160,0.831822,0.812975,0.810208,0.809933
1200,0.317300,0.421759,0.838246,0.818191,0.817574,0.817579
1400,0.296200,0.441385,0.842404,0.824745,0.825233,0.824566
1600,0.256500,0.464430,0.841270,0.823649,0.818027,0.819563
1800,0.216100,0.482335,0.837868,0.819402,0.816247,0.816086
2000,0.201500,0.471463,0.846183,0.828354,0.826324,0.827126


***** Running Evaluation *****
  Num examples = 2646
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-200
Configuration saved in ./results/checkpoint-200/config.json
Model weights saved in ./results/checkpoint-200/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2646
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-400
Configuration saved in ./results/checkpoint-400/config.json
Model weights saved in ./results/checkpoint-400/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2646
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-600
Configuration saved in ./results/checkpoint-600/config.json
Model weights saved in ./results/checkpoint-600/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2646
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-800
Configuration saved in ./results/checkpoint-800/config.json
Model weights saved in ./results/checkpoint-800/pytorch_model.bin
****

TrainOutput(global_step=3310, training_loss=0.31548456785542006, metrics={'train_runtime': 359.0169, 'train_samples_per_second': 294.805, 'train_steps_per_second': 9.22, 'total_flos': 1388122505280000.0, 'train_loss': 0.31548456785542006, 'epoch': 10.0})

In [ ]:
# 评估模型
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2646
  Batch size = 16


{'epoch': 10.0,
 'eval_F1': 0.8271258163432388,
 'eval_accuracy': 0.846182917611489,
 'eval_loss': 0.4714634120464325,
 'eval_precision': 0.8283544358562372,
 'eval_recall': 0.8263242868633087,
 'eval_runtime': 2.5871,
 'eval_samples_per_second': 1022.785,
 'eval_steps_per_second': 64.166}

In [ ]:
# 保存最好模型到指定位置
from random import randrange
import re
model_name = re.sub("/","_",model_name)
model_path = f"/content/drive/MyDrive/BERT_CLEANED_dataset/{model_name}"#保存路径
# model.save_pretrained(model_path)
# tokenizer.save_pretrained(model_path)

In [ ]:
# 重新加载训练好的模型和分词器
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(target_names))
tokenizer = BertTokenizerFast.from_pretrained(model_path)

NameError: ignored

In [ ]:
target_names = [0,1,2,3]
#分类标签函数
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=100, return_tensors="pt")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return target_names[probs.argmax()]


In [ ]:
from transformers import BertTokenizerFast, BertForSequenceClassification
model_path = '/content/drive/MyDrive/BERT_CLEANED_dataset/hfl_chinese-roberta-wwm-ext'

model = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(target_names))
tokenizer = BertTokenizerFast.from_pretrained(model_path)

In [ ]:
string = """高考的小朋友们要加油加油加油
高考加油！万事胜意！
少年们，高考加油！
加油吧，莘莘学子，奋斗三年，终于迎来了高考，祝你们一个月后金榜题

北约，罪恶滔天
我只记得那年美国大使馆院子里的石头、墨水瓶还有外墙上的鸡蛋液
北约，这个组织犯下的罪恶不能饶恕！

对隐瞒行程造成严重后果的必须严惩、严办。
何苦
损人不利己
为啥还有这种自私的人啊  隐瞒行程有什么好处吗？？？
各位共情怪请先把《中华人民共和国传染病防治法》了解一下，现在又不提依法治国了？
可恶
这种隐瞒行程的行为可耻，如果封控有相应生活补贴，估计隐瞒行程的会少很多，谁也不想自己的生活受影响，经济上受损失，这是人性，任何人都一样

什么时候才可以恢复正常生活，之前以为打了疫苗可以恢复正常生活，唉😔现在反反复复都不知何时是尽头，既然都这么严厉管控，为何不一起严管随地吐痰，及口罩统一消毒销毁处理，难得随地吐痰和佩戴过的口罩，都不会成为传播的途径？
可怜了动物们
请不要放过一个坏人北京加油
千万别隐瞒啊，害人害己。诚实上报，降低感染望周知"""
s_cheet =  string.splitlines()

In [ ]:
import re
#筛选评论只有表情的评论
def wash_dataset(clean_row):
    str1 = re.sub('\[[\u4e00-\u9fa5]+\]','',str(clean_row))
    str1 = re.sub('[^\u4e00-\u9fa5^。^，]','',str(clean_row))
    return str1

#返回清洗过后评论的列表，清洗过后与原评论评论组成的键值对字典
def get_data(raw_string_list):
  comment_list = []
  for i in raw_string_list:
      comment_list.append(wash_dataset(i))
  wash_rawlist = dict(zip(comment_list,raw_string_list))
  return comment_list,wash_rawlist


In [ ]:
def classification(emotion_value,kv_dict,wash_rawlist):
  emotion_dict={0:'开心',1:'生气',2:'厌恶',3:'失落'}
  print('#########################################################')
  print(f"筛选的情感评论为：{emotion_dict.get(emotion_value)}结果如下")
  for key,value in kv_dict.items():
    if value == emotion_value:
      print(wash_rawlist.get(key))

In [ ]:
def wash_raw_dict(s_cheet):
  wash_s_cheet = []
  for i in s_cheet:
    wash_s_cheet.append(wash_dataset(i))
  wash_rawlist = dict(zip(wash_s_cheet,s_cheet))
  return wash_s_cheet

def wash_result_dict(wash_s_cheet,wash_rawlist):
  result = []
  for i in wash_s_cheet:
    result.append(get_prediction(i))
  s_dict = dict(zip(wash_s_cheet,result))
  for i in range(4):
    classification(i,s_dict,wash_rawlist)

comment_list,wash_rawlist = get_data(s_cheet)
    
wash_s_cheet = wash_raw_dict(s_cheet)

wash_result_dict(wash_s_cheet,wash_rawlist)

#########################################################
筛选的情感评论为：开心结果如下
高考的小朋友们要加油加油加油
高考加油！万事胜意！
少年们，高考加油！
加油吧，莘莘学子，奋斗三年，终于迎来了高考，祝你们一个月后金榜题
#########################################################
筛选的情感评论为：生气结果如下
北约，罪恶滔天
我只记得那年美国大使馆院子里的石头、墨水瓶还有外墙上的鸡蛋液
北约，这个组织犯下的罪恶不能饶恕！
#########################################################
筛选的情感评论为：厌恶结果如下

对隐瞒行程造成严重后果的必须严惩、严办。
何苦
损人不利己
为啥还有这种自私的人啊  隐瞒行程有什么好处吗？？？
各位共情怪请先把《中华人民共和国传染病防治法》了解一下，现在又不提依法治国了？
可恶
这种隐瞒行程的行为可耻，如果封控有相应生活补贴，估计隐瞒行程的会少很多，谁也不想自己的生活受影响，经济上受损失，这是人性，任何人都一样
#########################################################
筛选的情感评论为：失落结果如下
什么时候才可以恢复正常生活，之前以为打了疫苗可以恢复正常生活，唉😔现在反反复复都不知何时是尽头，既然都这么严厉管控，为何不一起严管随地吐痰，及口罩统一消毒销毁处理，难得随地吐痰和佩戴过的口罩，都不会成为传播的途径？
可怜了动物们
请不要放过一个坏人北京加油
千万别隐瞒啊，害人害己。诚实上报，降低感染望周知


In [ ]:
# wash_s = []
# for i in s:
#   wash_s.append(wash_dataset(str(i)))
# test_dict = dict(zip(wash_s,s))
# result_washs = concat_dict(wash_s)

In [ ]:
# classification(0,result_wash_s)
# classification(1,result_wash_s)
# classification(2,result_wash_s)
# classification(3,result_wash_s)

In [ ]:
#wandb.finish()